In [2]:
import numpy as np 
import pandas as pd
import urllib.request 
import requests, io
import urllib, re, string
import datetime

In [2]:
pd.set_option('display.notebook_repr_html', True)

def _repr_latex_(self):
    return "\centering{%s}" % self.to_latex()

pd.DataFrame._repr_latex_ = _repr_latex_  # monkey patch pandas DataFrame

Data Collected 1/23/2016  

https://urldefense.proofpoint.com/v2/url?u=http-3A__mint.bio.uniroma2.it_mitab_MINT-5FMiTab.txt&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=Bv-GMiHdQIC4fRHi4JERcK71UMdrk1EvTGoRMpquPiM&e=  (Mint) 

Data Collected 1/25/2016  

https://urldefense.proofpoint.com/v2/url?u=https-3A__thebiogrid.org_downloads_archives_Release-2520Archive_BIOGRID-2D3.4.144_BIOGRID-2DALL-2D3.4.144.mitab.zip&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=mI2tEi0k6-2vBnUAJ8GD2-TCPVdPWzpMbje_637goZs&e=  (BioGrid)  
https://urldefense.proofpoint.com/v2/url?u=http-3A__www.innatedb.com_download_interactions_innatedb-5Fppi.mitab.gz&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=M4I-F00djsSBy83R9CVNADb-OMwpiel94VVNTR-murY&e=  (InnateDB)  
https://urldefense.proofpoint.com/v2/url?u=http-3A__dip.doe-2Dmbi.ucla.edu_dip_File.cgi-3FFN-3D2016_tab25_dip20160731.txt&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=D7qKgclA4IbvfDjbxccL8LSG3UsPa7AU4ant8b8qXWo&e=  (DIP)

Data Collected 1/26/2016   
https://urldefense.proofpoint.com/v2/url?u=http-3A__mentha.uniroma2.it_doDownload.php-3Ffile-3Dorganisms_9606.zip&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=RMnXZik8FZh0xdEL4OZPLacO9h0pmWzSKsHvkFp3Jt4&e=  (mentha)  
https://urldefense.proofpoint.com/v2/url?u=http-3A__www.ebi.ac.uk_intact_downloads&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=F7QsaA5RcDLVV_g_n7uDXIrPdh-SJk3JYZjYAE8HzO4&e=  (IntAct)  

In [3]:
def concatliketerms(df):
    
    df.sort_values(by='Protein A (gene name)', inplace=True)    
    df.reset_index(inplace=True)
    df.drop('index', axis=1, inplace=True)
    oldSize = df.shape
    newSize = 0
    j = 1
    while oldSize != newSize:
        df.reset_index(inplace=True)
        df.drop('index', axis=1, inplace=True)
        df.sort_values(by='Protein A (gene name)', inplace=True) 
        # concattonate like terms and remove duplicates
        i = 1
        end = len(df.index)-1
        for index in df.index:
            if i % 100 == 0:
                print('\r', 'Loop '+ str(j)+': ' +str(i)+' Out of '+ str(len(df.index)), end=' ', flush=True)
            if index in df.index.values and index != end:
                if df.ix[index, 'Protein A (gene name)'] == df.ix[index+1, 'Protein A (gene name)']:
                    if df.ix[index, 'Protein B (gene name)'] == df.ix[index+1, 'Protein B (gene name)']:
                        df.ix[index, 'PubMed ID'] = str(df.ix[index, 'PubMed ID']) +'|'+str(df.ix[index+1, 'PubMed ID'])
                        df.ix[index, 'Source databases'] = df.ix[index, 'Source databases'] +'|'+df.ix[index+1, 'Source databases']
                        df.drop(index+1, axis=0, inplace=True)
            i += 1
        j += 1
        oldSize = newSize
        newSize = df.shape

In [4]:
# column identifiers for initial data (as taken from respective metadata)
col_name = ['Unique identifier for interactor A',
      'Unique identifier for interactor B',
      'Alternative identifier for interactor A',
      'Alternative identifier for interactor B',
      'Protein A (gene name)',
      'Protein B (gene name)',
      'Interaction detection methods',
      'First author',
      'PubMed ID',
      'NCBI Taxonomy identifier for interactor A',
      'NCBI Taxonomy identifier for interactor B',
      'Interaction types',
      'Source databases', 
      'Interaction identifier(s)',
      'Confidence score']

#### Read mint data from url and creat file

In [4]:
##read in data from Mint and write to file (only needs to be proformed once)
# content=urllib.request.urlopen('https://urldefense.proofpoint.com/v2/url?u=http-3A__mint.bio.uniroma2.it_mitab_MINT-5FMiTab.txt&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=Bv-GMiHdQIC4fRHi4JERcK71UMdrk1EvTGoRMpquPiM&e= ') 

# target = open('mint.tsv', 'wb')
# for line in content:
#     target.write(line)
# target.close()

## Mint Data

In [150]:
mint_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/mint.tsv', sep='\t', index_col=False, header=None)

In [151]:
# assign names to columns from meta data
for i in np.arange(len(col_name)):
    mint_df.rename(columns={i:col_name[i]}, inplace=True)

In [152]:
# get only human (or mouse) data
n = mint_df['Protein A (gene name)'].values
b = [i for i,item in enumerate(n) if "mouse" in item]
mint_df = mint_df.ix[b]

In [153]:
mint_df =  mint_df.reset_index().drop('index', axis=1)

# get only human (or mouse) data
n = mint_df['Protein B (gene name)'].values
b = [i for i,item in enumerate(n) if "mouse" in item]
mint_df = mint_df.ix[b]

In [154]:
# select only relevent data
mint_df = mint_df[['Protein A (gene name)', 'Protein B (gene name)', 'PubMed ID', 'Source databases']]

In [155]:
# change column one to just show gene name
#i = 1
lst = []
for name in mint_df['Protein A (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name)
    if '(gene name)' in name:
        temp = name.split('|')
        for ID in temp:
            if '(gene name)' in ID:
                lst.append(ID.split(':')[1].split('(')[0])
                break

    else:
        lst.append(np.nan)
    
    #i += 1
mint_df['Protein A (gene name)'] = lst

In [156]:
# change column two to just show gene name
#i = 1
lst = []
for name in mint_df['Protein B (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name)
    if '(gene name)' in name:
        temp = name.split('|')
        for ID in temp:
            if '(gene name)' in ID:
                lst.append(ID.split(':')[1].split('(')[0])
                break

    else:
        lst.append(np.nan)
    
    #i += 1
mint_df['Protein B (gene name)'] = lst

In [157]:
# drop data for which no gene name is provited (or ensamble ID)
mint_df.dropna(how='any', inplace=True, axis=0)

In [158]:
lst = []

for pub in mint_df['PubMed ID']:
    pub = str(pub)
    pub = pub.split('|')[1].split(':')[1]
    lst.append(pub)
mint_df['PubMed ID'] = lst

In [159]:
mint_df.drop_duplicates(inplace=True)

In [160]:
mint_uf_df = mint_df.copy()

In [161]:
concatliketerms(mint_uf_df)

 Loop 6: 9400 Out of 9471 

In [162]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mint_mouse_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
mint_df.to_csv(filename, sep='\t', index=False) 

In [163]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mint_mouse_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
mint_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

In [164]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
lst = []
for pub in mint_df['PubMed ID'].values:
    print('\r', str(i)+' Out of '+ str(len(mint_df['PubMed ID'].values)), end=' ', flush=True)
    lst.append(mint_df[mint_df['PubMed ID'] == pub].shape[0] <= 10)
    i +=1

mint_df = mint_df[lst]

 9510 Out of 9510 

In [165]:
# Drop data for which there is no PubMed ID
lst = []
for value in mint_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

mint_df = mint_df.drop(mint_df[lst].index)

In [166]:
# Drop any data missing information
mint_df.dropna(inplace=True)

In [167]:
concatliketerms(mint_df)

 Loop 5: 1500 Out of 1532 

In [168]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mint_mouse_filtered_%s.tsv'% strhttps://urldefense.proofpoint.com/v2/url?u=http-3A__-28datetime.date.today&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=0AXIrr2R1Ws2uctDrGSv2BrKSII9GfdeCfYYet-im-Y&e= ()).replace('-', '_')
mint_df.to_csv(filename, sep='\t', index=False) 

In [169]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mint_mouse_filtered_%s.tsv.gz'% strhttps://urldefense.proofpoint.com/v2/url?u=http-3A__-28datetime.date.today&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=0AXIrr2R1Ws2uctDrGSv2BrKSII9GfdeCfYYet-im-Y&e= ()).replace('-', '_')
mint_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

## BioGrid Data

In [27]:
biogrid_df =https://urldefense.proofpoint.com/v2/url?u=http-3A__pd.re&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=kUtB02_d29l-AX62DWgGfOrrMBgeV15cPbw0hAtCi7Y&e= ad_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/BIOGRID-ALL-3.4.144.mitab.txt', sep='\t', index_col=False)

In [28]:
# get only human (or mouse) data
n = biogrid_df['Taxid Interactor A'].values
b = [i for i,item in enumerate(n) if "taxid:10090" in item] #taxid:9606 for human, taxid:10090 for mouse
biogrid_df = biogrid_df.ix[b]

In [29]:
biogrid_df =  biogrid_df.reset_index().drop('index', axis=1)

# get only human (or mouse) data
n = biogrid_df['Taxid Interactor B'].values
b = [i for i,item in enumerate(n) if "taxid:10090" in item] #taxid:9606 for human, taxid:10090 for mouse
biogrid_df = biogrid_df.ix[b]

In [30]:
# select only relevent data
biogrid_df = biogrid_df[['Alt IDs Interactor A', 'Alt IDs Interactor B', 'Publication Identifiers', 'Source Database']]

In [31]:
# rename columns for consistency
biogrid_df.rename(columns={'Alt IDs Interactor A': 'Protein A (gene name)', 'Alt IDs Interactor B':'Protein B (gene name)', 'Publication Identifiers':'PubMed ID', 'Source Database':'Source databases'}, inplace=True)

In [32]:
# change column one to just show gene name
i = 1
lst = []
for name in biogrid_df['Protein A (gene name)']:
    if i % 100 == 0:
        print('\r', i, end=' ', flush=True)
    name = str(name) 
    if name != 'nan':
        lst.append(name.split('|')[1].split(':')[1])
    else:
        lst.appendhttps://urldefense.proofpoint.com/v2/url?u=http-3A__-28np.na&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=7WM6dIU5B2DmwOyR_-PTnzSdk60Ct63WnSExIongu-A&e= n)
    i += 1
biogrid_df['Protein A (gene name)'] = lst

 17700 

In [33]:
# change column two to just show gene name
i = 1
lst = []
for name in biogrid_df['Protein B (gene name)']:
    if i % 100 == 0:
        print('\r', i, end=' ', flush=True)
    name = str(name) 
    if name != 'nan':
        lst.append(name.split('|')[1].split(':')[1])
    else:
        lst.appendhttps://urldefense.proofpoint.com/v2/url?u=http-3A__-28np.na&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=7WM6dIU5B2DmwOyR_-PTnzSdk60Ct63WnSExIongu-A&e= n)
    i += 1
biogrid_df['Protein B (gene name)'] = lst

 17700 

In [34]:
# drop data for which no gene name is provited (or ensamble ID)
biogrid_df.dropna(how='any', inplace=True, axis=0)

In [35]:
# Get Only PubMed ID for publication identifier
lst = []

for pub in biogrid_df['PubMed ID']:
    pub = str(pub)
    pub = pub.split(':')[1]
    lst.append(pub)
biogrid_df['PubMed ID'] = lst

In [36]:
biogrid_df.drop_duplicates(inplace=True)

In [37]:
# uf = unfiltered
biogrid_uf_df = biogrid_df.copy()

In [38]:
concatliketerms(biogrid_uf_df)

 Loop 44: 15700 Out of 15719 

In [39]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_biogrid_mouse_unfiltered_%s.tsv'% strhttps://urldefense.proofpoint.com/v2/url?u=http-3A__-28datetime.date.today&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=0AXIrr2R1Ws2uctDrGSv2BrKSII9GfdeCfYYet-im-Y&e= ()).replace('-', '_')
biogrid_uf_df.to_csv(filename, sep='\t', index=False) 

In [40]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_biogrid_mouse_unfiltered_%s.tsv.gz'% strhttps://urldefense.proofpoint.com/v2/url?u=http-3A__-28datetime.date.today&d=DwIGaQ&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=0AXIrr2R1Ws2uctDrGSv2BrKSII9GfdeCfYYet-im-Y&e= ()).replace('-', '_')
biogrid_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip')

In [41]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
for pub in biogrid_df['PubMed ID'].unique():
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(biogrid_df['PubMed ID'].unique())), end=' ', flush=True)
    if biogrid_df[biogrid_df['PubMed ID'] == pub].shape[0] > 10:
        biogrid_df.drop(biogrid_df[biogrid_df['PubMed ID'] == pub].index.tolist(), inplace=True)
    i +=1

 2900 Out of 2775 

In [42]:
biogrid_df.shape

(7203, 4)

In [43]:
lst = []
for value in biogrid_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

biogrid_df = biogrid_df.drop(biogrid_df[lst].index)

In [44]:
biogrid_df.dropna(inplace=True)

In [45]:
concatliketerms(biogrid_df)

 Loop 41: 6600 Out of 6650 

In [46]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_biogrid_mouse_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
biogrid_df.to_csv(filename, sep='\t', index=False) 

In [47]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_biogrid_mouse_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
biogrid_df.to_csv(filename, sep='\t', index=False, compression='gzip')

## InnateDB Data

In [48]:
innatedb_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/innatedb_ppi.mitab', sep='\t', index_col=False)

In [49]:
# get only human (or mouse) data
n = innatedb_df['ncbi_taxid_A'].values
b = [i for i,item in enumerate(n) if "Mouse" in item]
innatedb_df = innatedb_df.ix[b]

In [50]:
innatedb_df =  innatedb_df.reset_index().drop('index', axis=1)

# get only human (or mouse) data
n = innatedb_df['ncbi_taxid_B'].values
b = [i for i,item in enumerate(n) if "Mouse" in item]
innatedb_df = innatedb_df.ix[b]

In [51]:
innatedb_df = innatedb_df[['alias_A', 'alias_B', 'pmid', 'source_database']]

In [52]:
innatedb_df.rename(columns={'alias_A': 'Protein A (gene name)', 'alias_B':'Protein B (gene name)', 'pmid':'PubMed ID', 'source_database':'Source databases'}, inplace=True)

In [53]:
# change column one to just show gene name
#i = 1
lst = []
for name in innatedb_df['Protein A (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name) 
    if name != 'nan':
        lst.append(name.split('|')[-1].split(":")[1].split('(')[0])
    else:
        lst.append(np.nan)
    #i += 1
innatedb_df['Protein A (gene name)'] = lst

In [54]:
# change column two to just show gene name
#i = 1
lst = []
for name in innatedb_df['Protein B (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name) 
    if name != 'nan':
        lst.append(name.split('|')[-1].split(":")[1].split('(')[0])
    else:
        lst.append(np.nan)
    #i += 1
innatedb_df['Protein B (gene name)'] = lst

In [55]:
# drop data for which no gene name is provited (or ensamble ID)
innatedb_df.dropna(how='any', inplace=True, axis=0)

In [56]:
lst = []

for pub in innatedb_df['PubMed ID']:
    pub = str(pub)
    pub = pub.split(':')[1]
    lst.append(pub)
innatedb_df['PubMed ID'] = lst

In [57]:
innatedb_df.drop_duplicates(inplace=True)

In [58]:
innatedb_uf_df = innatedb_df.copy()

In [59]:
concatliketerms(innatedb_uf_df)

 Loop 17: 2300 Out of 2360 

In [60]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_innatedb_mouse_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
innatedb_uf_df.to_csv(filename, sep='\t', index=False) 

In [61]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_innatedb_mouse_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
innatedb_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

In [62]:
innatedb_df.shape

(2585, 4)

In [63]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
for pub in innatedb_df['PubMed ID'].unique():
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(innatedb_df['PubMed ID'].unique())), end=' ', flush=True)
    if innatedb_df[innatedb_df['PubMed ID'] == pub].shape[0] > 10:
        innatedb_df.drop(innatedb_df[innatedb_df['PubMed ID'] == pub].index.tolist(), inplace=True)
    i +=1

 900 Out of 921 

In [64]:
lst = []
for value in innatedb_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

innatedb_df = innatedb_df.drop(innatedb_df[lst].index)

In [65]:
innatedb_df.dropna(inplace=True)

In [66]:
concatliketerms(innatedb_df)

 Loop 20: 1700 Out of 1711 

In [67]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_innatedb_mouse_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
innatedb_df.to_csv(filename, sep='\t', index=False) 

In [68]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_innatedb_mouse_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
innatedb_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

## DIP DATA

In [70]:
dip_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/dip20160731.txt', sep='\t', index_col=False)

In [71]:
# get only human (or mouse) data
n = dip_df['Taxid interactor A'].values
b = [i for i,item in enumerate(n) if "Mus musculus" in item]
dip_df = dip_df.ix[b]

In [72]:
dip_df =  dip_df.reset_index().drop('index', axis=1)

# get only human (or mouse) data
n = dip_df['Taxid interactor B'].values
b = [i for i,item in enumerate(n) if "Mus musculus" in item]
dip_df = dip_df.ix[b]

In [73]:
dip_df = dip_df[['ID interactor A', 'ID interactor B', 'Publication Identifier(s)', 'Source database(s)']]

In [74]:
dip_df.rename(columns={'ID interactor A': 'Protein A (gene name)', 'ID interactor B':'Protein B (gene name)', 'Publication Identifier(s)':'PubMed ID', 'Source database(s)':'Source databases'}, inplace=True)

#### mapping table to convert labels from uniprot to ncbi names

In [75]:
mapping_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Doc and Mapping/MOUSE_10090_idmapping.dat', sep='\t', index_col=False, names=['UniProtKB-AC', 'ID_type', 'ID'])

In [76]:
mapping_df = mapping_df[mapping_df['ID_type']=='Gene_Name']

In [77]:
mapping_df.set_index('UniProtKB-AC', inplace=True)

In [78]:
mapping_df = mapping_df[~mapping_df.index.duplicated(keep='first')]

In [79]:
# change column one to just show gene name
for index in dip_df.index:
    found = re.search('uniprotkb:......', dip_df.ix[index, 'Protein A (gene name)'])
    if found:
        name = found.group(0).split(':')[1]
        if name in mapping_df.index.values:
            dip_df.ix[index, 'Protein A (gene name)'] = mapping_df.ix[name, 'ID']
        else:
            dip_df.ix[index, 'Protein A (gene name)'] = np.nan
    else:
        dip_df.ix[index, 'Protein A (gene name)'] = np.nan

In [80]:
# change column two to just show gene name
for index in dip_df.index:
    found = re.search('uniprotkb:......', dip_df.ix[index, 'Protein B (gene name)'])
    if found:
        name = found.group(0).split(':')[1]
        if name in mapping_df.index.values:
            dip_df.ix[index, 'Protein B (gene name)'] = mapping_df.ix[name, 'ID']
        else:
            dip_df.ix[index, 'Protein B (gene name)'] = np.nan
    else:
        dip_df.ix[index, 'Protein B (gene name)'] = np.nan

In [81]:
lst = []

for pub in dip_df['PubMed ID']:
    pub = str(pub)
    pub = pub.split('|')[0].split(':')[1]
    lst.append(pub)
dip_df['PubMed ID'] = lst

In [82]:
dip_df.drop_duplicates(inplace=True)

In [83]:
dip_uf_df = dip_df.copy()

In [84]:
concatliketerms(dip_uf_df)

 Loop 2: 1400 Out of 1443 

In [85]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_dip_mouse_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
dip_uf_df.to_csv(filename, sep='\t', index=False) 

In [86]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_dip_mouse_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
dip_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

In [87]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
for pub in dip_df['PubMed ID'].unique():
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(dip_df['PubMed ID'].unique())), end=' ', flush=True)
    if dip_df[dip_df['PubMed ID'] == pub].shape[0] > 10:
        dip_df.drop(dip_df[dip_df['PubMed ID'] == pub].index.tolist(), inplace=True)
    i +=1

 600 Out of 626 

In [88]:
lst = []
for value in dip_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

dip_df = dip_df.drop(dip_df[lst].index)

In [89]:
dip_df.dropna(inplace=True)

In [69]:
#mapping_df.head()

In [91]:
concatliketerms(dip_df)

 Loop 2: 1200 Out of 1210 

In [92]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_dip_mouse_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
dip_df.to_csv(filename, sep='\t', index=False) 

In [93]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_dip_mouse_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
dip_df.to_csv(filename, sep='\t', index=False, compression='gzip')

## mentha data

In [94]:
mentha_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/mentha_mouse', sep=';')

In [95]:
mentha_df = mentha_df[['Gene A', 'Gene B', 'PMID']]

In [96]:
mentha_df['Source databases'] = '(mentha)'

In [97]:
mentha_df.rename(columns={'Gene A': 'Protein A (gene name)', 'Gene B':'Protein B (gene name)', 'PMID':'PubMed ID'}, inplace=True)

In [98]:
i = 1
for index in mentha_df.index:
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(mentha_df.index)), end=' ', flush=True)
    mentha_df.ix[index, 'PubMed ID'] = '|'.join(mentha_df.ix[index, 'PubMed ID'].split(' ')[0:-1])
    i +=1

 26200 Out of 26282 

In [99]:
mentha_df.drop_duplicates(inplace=True)

In [100]:
mentha_uf_df = mentha_df.copy()

In [101]:
concatliketerms(mentha_uf_df)

 Loop 4: 26100 Out of 26122 

In [102]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mentha_mouse_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
mentha_uf_df.to_csv(filename, sep='\t', index=False) 

In [103]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mentha_mouse_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
mentha_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip')

In [104]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
for pub in mentha_df['PubMed ID'].unique():
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(mentha_df['PubMed ID'].unique())), end=' ', flush=True)
    if mentha_df[mentha_df['PubMed ID'] == pub].shape[0] > 10:
        mentha_df.drop(mentha_df[mentha_df['PubMed ID'] == pub].index.tolist(), inplace=True)
    i +=1

 4200 Out of 4139 

In [105]:
lst = []
for value in mentha_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

mentha_df = mentha_df.drop(mentha_df[lst].index)

In [106]:
mentha_df.dropna(inplace=True)

In [107]:
mentha_df.shape

(7640, 4)

In [108]:
concatliketerms(mentha_df)

 Loop 4: 7600 Out of 7632 

In [109]:
mentha_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,1700006A11RIK,BMPR1B,15761153,(mentha)
1,A2NTY6,A2NTU7,17418792,(mentha)
2,A6NAR8,A6NAR8,23986251,(mentha)
3,ABCA1,SPTLC1,18484747,(mentha)
4,ABCB10,ABCB10,15215243,(mentha)


In [110]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mentha_mouse_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
mentha_df.to_csv(filename, sep='\t', index=False) 

In [111]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mentha_mouse_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
mentha_df.to_csv(filename, sep='\t', index=False, compression='gzip')

## IntAct Data 

In [112]:
intact_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/intact.txt', sep='\t')

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [82]:
#intact_negative_df = pd.read_csv('intact_negative.txt', sep='\t')

In [83]:
#intact_df = pd.concat([intact_df, intact_negative_df])

In [113]:
# get only human (or mouse) data
n = intact_df['Taxid interactor A'].values
b = [i for i,item in enumerate(n) if "mouse" in item]
intact_df = intact_df.ix[b]

In [114]:
intact_df =  intact_df.reset_index().drop('index', axis=1)

# get only human (or mouse) data
n = intact_df['Taxid interactor B'].values
b = [i for i,item in enumerate(n) if "mouse" in item]
intact_df = intact_df.ix[b]

In [86]:
#intact_df.ix[0, 'Taxid interactor B']

In [115]:
intact_df = intact_df[['Alias(es) interactor A', 'Alias(es) interactor B', 'Publication Identifier(s)', 'Source database(s)']]

In [116]:
intact_df.rename(columns={'Alias(es) interactor A': 'Protein A (gene name)', 'Alias(es) interactor B':'Protein B (gene name)', 'Publication Identifier(s)':'PubMed ID', 'Source database(s)':'Source databases'}, inplace=True)

In [117]:
# change column one to just show gene name
#i = 1
lst = []
for name in intact_df['Protein A (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name)
    if '(gene name)' in name:
        temp = name.split('|')
        for ID in temp:
            if '(gene name)' in ID:
                lst.append(ID.split(':')[1].split('(')[0])
                break

    else:
        lst.append(np.nan)
    
    #i += 1
intact_df['Protein A (gene name)'] = lst

In [118]:
# change column two to just show gene name
#i = 1
lst = []
for name in intact_df['Protein B (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name)
    if '(gene name)' in name:
        temp = name.split('|')
        for ID in temp:
            if '(gene name)' in ID:
                lst.append(ID.split(':')[1].split('(')[0])
                break

    else:
        lst.append(np.nan)
    
    #i += 1
intact_df['Protein B (gene name)'] = lst

In [119]:
# drop data for which no gene name is provited (or ensamble ID)
intact_df.dropna(how='any', inplace=True, axis=0)

In [120]:
lst = []

for pub in intact_df['PubMed ID']:
    pub = str(pub)
    pub = pub.split('|')
    for p in pub:
        if 'pubmed' in p:
            pub = p.split(':')[1]
    lst.append(pub)
intact_df['PubMed ID'] = lst

In [121]:
intact_df.drop_duplicates(inplace=True)

In [122]:
intact_uf_df = intact_df.copy()

In [123]:
concatliketerms(intact_uf_df)

 Loop 25: 18800 Out of 18832 

In [124]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_intact_mouse_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
intact_uf_df.to_csv(filename, sep='\t', index=False) 

In [125]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_intact_mouse_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
intact_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip')

In [126]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
for pub in intact_df['PubMed ID'].unique():
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(intact_df['PubMed ID'].unique())), end=' ', flush=True)
    if intact_df[intact_df['PubMed ID'] == pub].shape[0] > 10:
        intact_df.drop(intact_df[intact_df['PubMed ID'] == pub].index.tolist(), inplace=True)
    i +=1


 1700 Out of 1662 

In [127]:
intact_df.shape

(4193, 4)

In [128]:
lst = []
for value in intact_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

intact_df = intact_df.drop(intact_df[lst].index)

In [129]:
intact_df.dropna(inplace=True)

In [130]:
intact_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
1,Braf,Ywhah,8668348,"psi-mi:""MI:0471""(MINT)"
3,Braf,Hras,8668348,"psi-mi:""MI:0471""(MINT)"
4,Map2k1,Braf,8668348,"psi-mi:""MI:0471""(MINT)"
5,Braf,Map2k2,8668348,"psi-mi:""MI:0471""(MINT)"
9,Adam12,Grb2,11127814,"psi-mi:""MI:0471""(MINT)"


In [99]:
#intact_negative_df.head()

In [100]:
# Alt. ID(s) interactor A
#intact_df.ix[22, 'Aliases for A']

In [131]:
concatliketerms(intact_df)

 Loop 13: 3900 Out of 3997 

In [132]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_intact_mouse_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
intact_df.to_csv(filename, sep='\t', index=False) 

In [133]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_intact_mouse_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
intact_df.to_csv(filename, sep='\t', index=False, compression='gzip')

### Data Tables

In [170]:
mint_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,Abcb7,Fech,fj-09-0740,"psi-mi:""MI:0471""(MINT)"
1,Abcb7,Cox4i1,fj-09-0740,"psi-mi:""MI:0471""(MINT)"
2,Abcb7,Cyb5r3,fj-09-0740,"psi-mi:""MI:0471""(MINT)"
3,Abl1,Abi1,11163214,"psi-mi:""MI:0471""(MINT)"
4,Abl1,Sh3bp2,MINT-5220831,"psi-mi:""MI:0471""(MINT)"


In [148]:
biogrid_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,AF251705,Hcst,12893283,"psi-mi:""MI:0463""(biogrid)"
1,AF251705,Tyrobp,12893283,"psi-mi:""MI:0463""(biogrid)"
2,AF251705,Fcer1g,12893283,"psi-mi:""MI:0463""(biogrid)"
3,AI464131,Igf2,19706595,"psi-mi:""MI:0463""(biogrid)"
4,Aatf,Maged1,17488777,"psi-mi:""MI:0463""(biogrid)"


In [147]:
innatedb_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,1700021K19Rik,Cyba,22423966,MI:0974(innatedb)
1,1700021K19Rik,Cybb,22423966,MI:0974(innatedb)
2,1700021K19Rik,Becn1,19270693,MI:0974(innatedb)
3,Abl1,Jun,19818398,MI:0974(innatedb)
4,Ackr2,Ccr3,9139699,MI:0974(innatedb)


In [146]:
dip_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,Abcc8,Ank2,19805355,MI:0465(dip)
1,Abhd5,Pnpla2,25961502,MI:0465(dip)
2,Abi2,Eps8,19564905,MI:0465(dip)
3,Acad9,Ecsit,21525932,MI:0465(dip)
4,Acd,Pot1b,17632522,MI:0465(dip)


In [145]:
mentha_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,1700006A11RIK,BMPR1B,15761153,(mentha)
1,A2NTY6,A2NTU7,17418792,(mentha)
2,A6NAR8,A6NAR8,23986251,(mentha)
3,ABCA1,SPTLC1,18484747,(mentha)
4,ABCB10,ABCB10,15215243,(mentha)


In [144]:
intact_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,Aatk,Stk39,14563843,"psi-mi:""MI:0469""(IntAct)"
1,Abca1,Nr1h2,23931754,"psi-mi:""MI:1332""(bhf-ucl)"
2,Abca1,Abca12,23931754,"psi-mi:""MI:1332""(bhf-ucl)"
3,Abcb7,Fech,19691493,"psi-mi:""MI:0471""(MINT)"
4,Abcb7,Cyb5r3,19691493,"psi-mi:""MI:0471""(MINT)"


## Comparison and overlap

In [135]:
data_sets= [mint_df, biogrid_df, innatedb_df, dip_df, mentha_df, intact_df]
data_sets_names= ['mint', 'biogrid', 'innatedb', 'dip', 'mentha', 'intact']

comp_df = pd.DataFrame(columns=data_sets_names, index=data_sets_names)
comp_df.columns.name = 'Overlap (Mouse)'
for i in np.arange(len(data_sets)):
    for j in np.arange(len(data_sets)):
        comp = pd.concat([data_sets[i][['Protein A (gene name)', 'Protein B (gene name)']], data_sets[j][['Protein A (gene name)', 'Protein B (gene name)']]])
        comp_df.ix[data_sets_names[i], data_sets_names[j]] = comp.duplicated().sum()

# mint_df[['Aliases for A', 'Aliases for B']].duplicated().sum()
# biogrid_df[['Aliases for A', 'Aliases for B']].duplicated().sum()
comp_df

Overlap (Mouse),mint,biogrid,innatedb,dip,mentha,intact
mint,1543,312,120,52,18,1571
biogrid,312,6757,434,276,114,753
innatedb,120,434,1765,114,61,281
dip,52,276,114,1211,8,180
mentha,18,114,61,8,7639,70
intact,1571,753,281,180,70,4060


## Merged Network Table

In [171]:
mint_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mint_mouse_filtered_2017_02_15.tsv', sep='\t')
biogrid_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_biogrid_mouse_filtered_2017_02_15.tsv', sep='\t')
innatedb_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_innatedb_mouse_filtered_2017_02_15.tsv', sep='\t')
dip_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_dip_mouse_filtered_2017_02_15.tsv', sep='\t')
mentha_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mentha_mouse_filtered_2017_02_15.tsv', sep='\t')
intact_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_intact_mouse_filtered_2017_02_15.tsv', sep='\t')

In [172]:
network_df = pd.concat([mint_df, biogrid_df, innatedb_df, dip_df, mentha_df, intact_df])

In [111]:
#network_df.rename(columns={'Aliases for A':'Protein A (gene name)', 'Aliases for B':'Protein B (gene name)', 'Identifier of the publication':'PubMed ID'}, inplace=True)

In [173]:
network_df.sort_values(['Protein A (gene name)', 'Protein B (gene name)'], inplace=True)

In [174]:
network_df.reset_index(inplace=True)
network_df.drop('index', axis=1, inplace=True)

In [175]:
network_df.shape

(22732, 4)

In [176]:
concatliketerms(network_df)

 Loop 46: 19800 Out of 19866 

In [180]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/Combined/Filtered/ppi_network_mouse_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
network_df.to_csv(filename, sep='\t', index=False) 

In [181]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/Combined/Filtered/ppi_network_mouse_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
network_df.to_csv(filename, sep='\t', index=False, compression='gzip')

In [179]:
network_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,1700006A11RIK,BMPR1B,15761153,(mentha)
1,1700021K19Rik,Becn1,19270693,MI:0974(innatedb)
2,1700021K19Rik,Cyba,22423966,MI:0974(innatedb)
3,1700021K19Rik,Cybb,22423966,MI:0974(innatedb)
4,A2NTY6,A2NTU7,17418792,(mentha)


In [182]:
mint_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mint_mouse_unfiltered_2017_02_15.tsv', sep='\t')
biogrid_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_biogrid_mouse_unfiltered_2017_02_15.tsv', sep='\t')
innatedb_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_innatedb_mouse_unfiltered_2017_02_15.tsv', sep='\t')
dip_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_dip_mouse_unfiltered_2017_02_15.tsv', sep='\t')
mentha_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mentha_mouse_unfiltered_2017_02_15.tsv', sep='\t')
intact_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_intact_mouse_unfiltered_2017_02_15.tsv', sep='\t')

In [183]:
network_uf_df = pd.concat([mint_uf_df, biogrid_uf_df, innatedb_uf_df, dip_uf_df, mentha_uf_df, intact_uf_df])

In [184]:
network_uf_df.sort_values(['Protein A (gene name)', 'Protein B (gene name)'], inplace=True)

In [185]:
network_uf_df.reset_index(inplace=True)
network_uf_df.drop('index', axis=1, inplace=True)

In [186]:
concatliketerms(network_uf_df)

 Loop 74: 61900 Out of 61963 

In [187]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/Combined/Unfiltered/ppi_network_mouse_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
network_uf_df.to_csv(filename, sep='\t', index=False) 

In [188]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/Combined/Unfiltered/ppi_network_mouse_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
network_df.to_csv(filename, sep='\t', index=False, compression='gzip')

## Merged DataFrame 

In [145]:
# drop PPI to self for the DIP data 
for index in dip_df.index:
    if dip_df.ix[index, 'Aliases for A'] == dip_df.ix[index, 'Aliases for B']:
        dip_df.drop(index, inplace=True)

In [146]:
df = pd.concat([mint_df, biogrid_df, innatedb_df, dip_df, mentha_df, intact_df])

In [147]:
df = df[['Aliases for A', 'Aliases for B']]

In [148]:
df.shape

(20962, 2)

In [149]:
df.drop_duplicates(['Aliases for A', 'Aliases for B'], inplace=True)

In [150]:
df.dropna(inplace=True)

In [151]:
df.shape

(18704, 2)

In [152]:
df.head()

,Aliases for A,Aliases for B
0,Rgs12,Dlgap3
1,Rgs12,Map2k2
2,Epn1,Itsn1
7,Rgs12,Araf
9,Znf496,Jarid2


## Gene Set Library

In [153]:
# build library

i = 1

genelist = set(df['Aliases for A'].values.tolist() + df['Aliases for B'].values.tolist())

lib = pd.DataFrame()

for gene in genelist:
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(genelist)), end=' ', flush=True)
    lst = set(df[df.values == gene].values.flatten().tolist())
    lst.remove(gene)
    lst = list(lst)
    lst.insert(0, gene)
    if len(lst) >= 50:
        temp = pd.DataFrame(data=lst)
        lib = pd.concat([lib, temp.T])
    i += 1

 8300 Out of 8354 

In [154]:
lib.to_csv('PPI_MOUSE.gmt', sep='\t', header=None, index=False)

In [109]:
lib.head(20)

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,Src,S1pr1,Ptk2,Diap1,Bcar1,Traf6,Shc1,Akt1,Pdcd6ip,Csn2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,HDAC1,EZH2,BRMS1,ETS1,MAD1L1,RCOR1,EED,PPIA,WHSC1,HNF1B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Hdac1,Ccne1,Sp3,Brms1,Jdp2,Pdx1,Ifrd1,Mad1l1,Smyd1,Kdm1a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Pik3r1,Ptk2,Ncor1,Ank3,Klrk1,Thrb,Tnfrsf11a,Ptpn11,Ep300,Akt1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,TP53,PIAS2,MAPK8,PIN1,UBE2N,GSK3B,ZBTB7C,RPS26,NDN,BACH1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Cbl,Spta1,Sh2b2,Fgr,Zap70,Dok3,Src,Pik3r1,Traf6,Tnfrsf11a,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NANOG,MAPK1,PIN1,POGZ,WHSC1,MAPK3,SMARCA2,KDM1A,ZMYM2,TRIM28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,CTNNB1,HTT,MAPK8,LEF1,PIN1,TNIK,GSK3B,JUN,CTNNA1,KIFAP3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,TRAF6,UBE2D3,UBE2N,MAP3K7,SQSTM1,RIPK1,TNFRSF19,TAB2,MYSM1,UBC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Smarca4,Smarcc1,Klf4,Nf1,Arid1b,Polr2a,Actl6b,Nr1h4,Hdac3,Nkx2-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [220]:
lib.shape

(392, 589)

## Protein Count

In [4]:
mint_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_mint_mouse_filtered_2017_02_15.tsv', sep='\t')
biogrid_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_biogrid_mouse_filtered_2017_02_15.tsv', sep='\t')
innatedb_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_innatedb_mouse_filtered_2017_02_15.tsv', sep='\t')
dip_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_dip_mouse_filtered_2017_02_15.tsv', sep='\t')
mentha_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_mentha_mouse_filtered_2017_02_15.tsv', sep='\t')
intact_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_intact_mouse_filtered_2017_02_15.tsv', sep='\t')

In [5]:
len(set(mint_df['Protein A (gene name)'].tolist()+mint_df['Protein B (gene name)'].tolist()))

1232

In [6]:
len(set(biogrid_df['Protein A (gene name)'].tolist()+biogrid_df['Protein B (gene name)'].tolist()))

3100

In [7]:
len(set(innatedb_df['Protein A (gene name)'].tolist()+innatedb_df['Protein B (gene name)'].tolist()))

1011

In [8]:
len(set(dip_df['Protein A (gene name)'].tolist()+dip_df['Protein B (gene name)'].tolist()))

1135

In [9]:
len(set(mentha_df['Protein A (gene name)'].tolist()+mentha_df['Protein B (gene name)'].tolist()))

3901

In [10]:
len(set(intact_df['Protein A (gene name)'].tolist()+intact_df['Protein B (gene name)'].tolist()))

2352